In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
df = pd.read_csv('previsoes_falhas_threshold_ajustado.csv')
df.head()

Type  Air_temperature_K  Process_temperature_K  Rotational_speed_rpm  \
0    L              300.5                  309.8                  1345   
1    L              303.7                  312.4                  1513   
2    L              302.5                  311.4                  1559   
3    H              295.6                  306.3                  1509   
4    H              300.5                  310.0                  1358   

   Torque_Nm  Tool_wear_min  Temp_diff  Torque_per_rpm  Real  Predito  \
0       62.7            153        9.3        0.046617     0        1   
1       40.1            135        8.7        0.026504     0        0   
2       37.6            209        8.9        0.024118     0        0   
3       35.8             60       10.7        0.023724     0        0   
4       60.4            102        9.5        0.044477     0        1   

   Prob_Falha  
0    0.938798  
1    0.434264  
2    0.543648  
3    0.044053  
4    0.844923

### PREPARAR OS DADOS

---



In [ ]:
df['Status Previsto'] = df['Predito'].map({0: 'Sem falha prevista', 1: 'Falha prevista'})
df['Status Previsto'].value_counts()

Status Previsto
Sem falha prevista    1928
Falha prevista          72
Name: count, dtype: int64

### DISTRIBUIÇÃO DAS PREVISÕES DE FALHAS

---



In [ ]:
fig1 = px.histogram(
    df,
    x='Status Previsto',
    color='Status Previsto',
    title='Distribuição das Previsões de Falhas'
)
fig1.update_layout(bargap=0.2)
fig1.show()

### PROBABILIDADE DE FALHA X TORQUE (Nm)

---



In [ ]:
fig2 = px.scatter(
    df,
    x='Torque_Nm',
    y='Prob_Falha',
    color='Status Previsto',
    title='Probabilidade de Falha x Torque (Nm)',
    hover_data=['Air_temperature_K', 'Process_temperature_K', 'Rotational_speed_rpm']
)
fig2.show()

### REAL vs PREDITO

---



In [ ]:
fig3 = px.histogram(
    df,
    x='Real',
    color='Predito',
    title='Distribuição Real x Predito'
)
fig3.show()

### MATRIZ DE CONFUSÃO ABSOLUTA

---



In [ ]:
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff


cm = confusion_matrix(df['Real'], df['Predito'])
cm_labels = ['Sem falha', 'Falha']

In [ ]:
fig_cm = ff.create_annotated_heatmap(
    z=cm,
    x=cm_labels,
    y=cm_labels,
    colorscale='Blues',
    showscale=True
)
fig_cm.update_layout(
    title_text='Matriz de Confusão (Contagem)',
    xaxis_title='Previsto',
    yaxis_title='Real'
)
fig_cm.show()


### MATRIZ DE CONFUSÃO NORMALIZADA

---



In [ ]:
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100

fig_cm_norm = ff.create_annotated_heatmap(
    z=np.round(cm_normalized, 1),
    x=cm_labels,
    y=cm_labels,
    colorscale='Greens',
    showscale=True
)
fig_cm_norm.update_layout(
    title_text='Matriz de Confusão Normalizada (%)',
    xaxis_title='Previsto',
    yaxis_title='Real'
)
fig_cm_norm.show()
